In [1]:
import os
import random
import math
import pygame
from os import listdir
from os.path import isfile, join
pygame.init()

pygame.display.set_caption("Block Runner")
#dislaying the name of the game on the top of the screen

WIDTH, HEIGHT = 800, 600
FPS = 60
PLAYER_VEL = 5
SCROLL = False

window = pygame.display.set_mode((WIDTH, HEIGHT))

def flip(sprites):
    return [pygame.transform.flip(sprite, True, False) for sprite in sprites]
    #using function to flip sprites horizontally and not vertically

def load_sprite_sheets(dir1, dir2, width, height, direction = False):

    #creating a path for the sprite sheets 
    path = join("assets", dir1, dir2)
    
    #(f for f in ...) is a comprehensive way to make a list out of every element f in something 
    #listdir(path) lists all the items(files and directories) in a specified directory 
    #we are checking if the item is a file using isfile()
    #we are creating the full path to the item using join()
    #finally we have a list of all the files in the directory
    images = [f for f in listdir(path) if isfile(join(path, f))]

    #initializing dictionary to store sprite data 
    all_sprites = {}

    #looping through each image file 
    for image in images:
        #loading the sprite sheet image 
        #convert alpha converts the loaded image to format that supports transparency
        sprite_sheet = pygame.image.load(join(path, image)).convert_alpha()

        #initialize a list to store individual sprites 
        sprites = []

        #we are cutting the sprite sheet into individual sprites of required height and width
        for i in range(sprite_sheet.get_width() // width):
            #creating a transparent surface for individual sprites 
            #SRCALPHA is a flag indicating that the surface will have an alpha channel
            surface = pygame.Surface((width, height), pygame.SRCALPHA, 32)
            #defining a rectangular region in the sprite sheet to extract
            rect = pygame.Rect(i * width, 0, width, height)
            #copy the defined region into the transparent surface 
            surface.blit(sprite_sheet, (0, 0), rect)
            #scale the size of the sprite by a factor of 2 and add it to the list 
            sprites.append(pygame.transform.scale2x(surface))

        #if direction is true we create both left and right versions of sprite sheets
        if direction:
            all_sprites[image.replace(".png", "") + "_right"] = sprites
            all_sprites[image.replace(".png", "") + "_left"] = flip(sprites)

        else:
            all_sprites[image.replace(".png", "")] = sprites

    return all_sprites   


#think about this again
def get_block(size):
    path = join("assets", "Terrain", "Terrain.png")
    image = pygame.image.load(path).convert_alpha()
    surface = pygame.Surface((size, size), pygame.SRCALPHA, 32)
    rect = pygame.Rect(96, 0, size, size)
    surface.blit(image, (0, 0), rect)
    return pygame.transform.scale2x(surface)

class Player(pygame.sprite.Sprite):
    COLOR = (255, 0, 0)
    GRAVITY = 1
    SPRITES = load_sprite_sheets("MainCharacters", "MaskDude", 32, 32, True)
    ANIMATION_DELAY = 5
    
    def __init__(self, x, y, width, height):
        super().__init__()
        #calling the constructor of superclass pygame.sprite.Sprite above 
        #ensures player object inherits the properties and behaviours of sprite class 
        self.rect = pygame.Rect(x, y, width, height)
        #representing the position of the player and dimensions of the player 
        self.x_vel = 0
        self.y_vel = 0
        self.mask = None
        #mask is used for collision detection
        self.direction = "left"
        self.animation_count = 0
        self.fall_count = 0
        self.jump_count = 0

    def jump(self):
        #setting the upward velocity of jump to about 8 times of the gravity of player 
        self.y_vel = -self.GRAVITY * 8
        #resetting animation count to zero but I'm not exactly sure why
        self.animation_count = 0
        #%keeping track of how many times player jumped 
        self.jump_count += 1
        #setting the fall count to zero when player is jumping
        if self.jump_count == 1:
            self.fall_count = 0

    
    def move(self, dx, dy):
        #moving the player along the screen 
        self.rect.x += dx
        self.rect.y += dy

    def move_left(self, vel):
        self.x_vel = -vel
        if self.direction != "left":
            self.direction = "left"
            self.animation_count = 0

    def move_right(self, vel):
        self.x_vel = vel
        if self.direction != "right":
            self.direction = "right"
            self.animation_count = 0

    def loop(self, fps):
        #self.fall_count / fps calculates the rate of falling 
        #this rate is multiplied with self.Gravity to replicate the gravity 
        #min so that it doesn't accelerate too fast and then we update the y velocity of the sprite 
        self.y_vel += min(1, (self.fall_count / fps) * self.GRAVITY)
        self.move(self.x_vel, self.y_vel)

        self.fall_count += 1
        #keeps track of for how many frames the player is falling
        self.update_sprite()

    def update_sprite(self):
        #determining the appropriate sprite sheet according to the player's movement 
        sprite_sheet = "idle"
        if self.x_vel != 0:
            sprite_sheet = "run"

        #initializing a sprite sheet name 
        sprite_sheet_name = sprite_sheet + "_" + self.direction
        #extarcing the desired sprite 
        sprites = self.SPRITES[sprite_sheet_name]
        #finding out the index of the sprite required so that it loops through the sprites 
        sprite_index = (self.animation_count // self.ANIMATION_DELAY) % len(sprites)
        self.sprite = sprites[sprite_index]
        self.animation_count += 1
        self.update()

    def update(self):
        #updating the postition of the sprite 
        #setting the topleft of the rectangle to the current position of the player 
        self.rect = self.sprite.get_rect(topleft=(self.rect.x, self.rect.y))
        #updates the mask used for collision detection 
        self.mask = pygame.mask.from_surface(self.sprite)
        
    def landed(self):
        #reset counters and velocity when the sprite lands on a surface
        self.fall_count = 0
        self.y_vel = 0
        self.jump_count = 0

    def hit_head(self):
        self.count = 0
        self.y_vel *= -1
    
    def draw(self, win):
        self.sprite = self.SPRITES["idle_" + self.direction][0]
        win.blit(self.sprite, (self.rect.x, self.rect.y))


class FallingBlock(pygame.sprite.Sprite):
    COLOR = (0, 255, 0)
    GRAVITY = 0.5
    SPRITES = load_sprite_sheets("Terrain", "Terrain", 32, 32, True)
    ANIMATION_DELAY = 5
    
    def __init__(self, x, y, width, height):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height)
        self.x_vel = 0
        self.y_vel = 0
        self.mask = None
        self.direction = "left"
        self.animation_count = 0
        self.fall_count = 0
        self.jump_count = 0

    def jump(self):
        self.y_vel = -self.GRAVITY * 8
        self.animation_count = 0
        self.jump_count += 1
        if self.jump_count == 1:
            self.fall_count = 0

    
    def move(self, dx, dy):
        self.rect.x += dx
        self.rect.y += dy

    def move_left(self, vel):
        self.x_vel = -vel
        if self.direction != "left":
            self.direction = "left"
            self.animation_count = 0

    def move_right(self, vel):
        self.x_vel = vel
        if self.direction != "right":
            self.direction = "right"
            self.animation_count = 0

    def loop(self, fps):
        #self.y_vel += min(0.2, (self.fall_count / fps) * self.GRAVITY)
        self.y_vel += 0.2
        self.move(self.x_vel, self.y_vel)

        self.fall_count += 1
        self.update_sprite()

    def update_sprite(self):
        sprite_sheet = "idle"
        if self.x_vel != 0:
            sprite_sheet = "run"

        sprite_sheet_name = sprite_sheet + "_" + self.direction
        sprites = self.SPRITES[sprite_sheet_name]
        sprite_index = (self.animation_count // self.ANIMATION_DELAY) % len(sprites)
        self.sprite = sprites[sprite_index]
        self.animation_count += 1
        self.update()

    def update(self):
        self.rect = self.sprite.get_rect(topleft=(self.rect.x, self.rect.y))
        self.mask = pygame.mask.from_surface(self.sprite)
        
    def landed(self):
        self.fall_count = 0
        self.y_vel = 0
        self.jump_count = 0

    def hit_head(self):
        self.count = 0
        self.y_vel *= -1
    
    def draw(self, win):
        self.sprite = self.SPRITES["idle_" + self.direction][0]
        win.blit(self.sprite, (self.rect.x, self.rect.y))


class Object(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height, name = None):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height)
        self.image = pygame.Surface((width, height), pygame.SRCALPHA)
        self.width = width
        self.height = height
        self.name = name

    def draw(self, win):
        win.blit(self.image, (self.rect.x, self.rect.y))

class Block(Object):
    def __init__(self, x, y, size):
        super().__init__(x, y, size, size)
        block = get_block(size)
        self.image.blit(block, (0, 0))
        self.mask = pygame.mask.from_surface(self.image)

def generate_tetris_block(size):
    x = random.randint(0, WIDTH - (4 * size))
    style = random.randint(1,8)
    if style == 1:
        # L block
        return [[FallingBlock(x, 65, 50, 50), False],
                [FallingBlock(x + 65, 65, 50, 50), False],
                [FallingBlock(x + 2 * 65, 65, 50, 50), False],
                [FallingBlock(x + 2 * 65, 0, 50, 50), False]]

    elif style == 2:
        # I block
        return [[FallingBlock(x, 0, 50, 50), False],
                [FallingBlock(x + 65, 0, 50, 50), False],
                [FallingBlock(x + 2 * 65, 0, 50, 50), False],
                [FallingBlock(x + 3 * 65, 0, 50, 50), False]]

    elif style == 3:
        # J block
         return [[FallingBlock(x, 0, 50, 50), False],
                [FallingBlock(x + 65, 65, 50, 50), False],
                [FallingBlock(x + 65, 65, 50, 50), False],
                [FallingBlock(x + 2 * 65, 65, 50, 50), False]]

    elif style == 4:
        # O block
         return [[FallingBlock(x, 0, 50, 50), False],
                [FallingBlock(x + 65, 0, 50, 50), False],
                [FallingBlock(x, 65, 50, 50), False],
                [FallingBlock(x + 65, 65, 50, 50), False]] 

    elif style == 5:
        # S block
         return [[FallingBlock(x + 65, 0, 50, 50), False],
                [FallingBlock(x + 2 * 65, 0, 50, 50), False],
                [FallingBlock(x, 65, 50, 50), False],
                [FallingBlock(x + 65, 65, 50, 50), False]]

    elif style == 6:
        # T block
         return [[FallingBlock(x, 65, 50, 50), False],
                [FallingBlock(x + 65, 65, 50, 50), False],
                [FallingBlock(x + 2 * 65, 65, 50, 50), False],
                [FallingBlock(x + 65, 0, 50, 50), False]]
            
    else:
        # Z block
         return [[FallingBlock(x, 0, 50, 50), False],
                [FallingBlock(x + 65, 0, 50, 50), False],
                [FallingBlock(x + 65, 65, 50, 50), False],
                [FallingBlock(x + 2 * 65, 65, 50, 50), False]]
    

def get_background(name): 
    image = pygame.image.load(join("assets", "Background", name))
    _, _, width, height = image.get_rect()
    tiles = []

    for i in range(WIDTH // width + 1):
        #horizontal 
        for j in range(HEIGHT // height + 1):
            #vertical
            pos = (i * width, j * height)
            tiles.append(pos)

    return tiles, image
    

# def draw(window, background, bg_image, player, objects, objects2):
#     for tile in background: 
#         window.blit(bg_image, tile)

#     for obj in objects:
#         obj.draw(window)

#     for obj in objects2:
#         obj.draw(window)

#     player.draw(window)

#     pygame.display.update()


def get_background(name): 
    image = pygame.image.load(join("assets", "Background", name))
    _, _, width, height = image.get_rect()
    tiles = []

    for i in range(WIDTH // width + 1):
        for j in range(HEIGHT // height + 1):
            pos = (i * width, j * height)
            tiles.append(pos)

    return tiles, image

def draw(window, background, bg_image, player, objects, objects2, instructions = False):
    for tile in background: 
        window.blit(bg_image, tile)

    for obj in objects:
        obj.draw(window)

    for obj in objects2:
        obj.draw(window)

    player.draw(window)

    if instructions:
        font = pygame.font.Font(None, 36)
        # text = font.render("Instructions: Use arrow keys to move. Space to jump. Press Esc to go back", True, (255, 255, 255))
        # window.blit(text, (WIDTH // 2 - text.get_width() // 2, HEIGHT // 2 - text.get_height() // 2))

    pygame.display.update()

def handle_vertical_collision(player, objects, dy):
    #initializing a list of collided objects 
    collided_objects = []

    #looping through each element in objects
    for obj in objects: 
        #checking if the player collided with the object
        if pygame.sprite.collide_mask(player, obj):
            #checking if the player was moving downward 
            if dy > 0:
                #setting the player to be on top of the surface
                player.rect.bottom = obj.rect.top
                player.landed()
            #checking if the player was moving upward
            elif dy < 0:
                player.rect.top = obj.rect.bottom
                player.hit_head()

        #adding the element to collided objects
        collided_objects.append(obj)

    return collided_objects
    

def collide(player, objects, dx):
    #temporarily moving player in small displacement 
    player.move(dx, 0)
    #updating the position of the player 
    player.update()
    collided_object = None 

    #checking if the player collided with any object 
    for obj in objects :
        if pygame.sprite.collide_mask(player, obj): 
            collided_object = obj
            break 

    #moving player back to it's original position 
    player.move(-dx, 0)
    player.update()
    return collided_object
                
def handle_move(player, objects):
    keys = pygame.key.get_pressed()

    player.x_vel = 0
    #checking for left and right collision 
    collide_left = collide(player, objects, -PLAYER_VEL * 2)
    collide_right = collide(player, objects, PLAYER_VEL * 2)
    
    if keys[pygame.K_a] and not collide_left:
        player.move_left(PLAYER_VEL)
    if keys[pygame.K_d] and not collide_right:
        player.move_right(PLAYER_VEL)

    handle_vertical_collision(player, objects, player.y_vel)

def handle_move_forFallingBlock(player, objects):
    keys = pygame.key.get_pressed()

    player[0].x_vel = 0
    collide_left = collide(player[0], objects, -PLAYER_VEL * 2)
    collide_right = collide(player[0], objects, PLAYER_VEL * 2)
    
    if keys[pygame.K_LEFT] and not collide_left:
        player[0].move_left(PLAYER_VEL)
    if keys[pygame.K_RIGHT] and not collide_right:
        player[0].move_right(PLAYER_VEL)

    handle_vertical_collision_forFallingBlock(player, objects, player[0].y_vel)
    
def handle_vertical_collision_forFallingBlock(player, objects, dy):
    collided_objects = []
    for obj in objects: 
        if pygame.sprite.collide_mask(player[0], obj):
            if dy > 0:
                player[0].rect.bottom = obj.rect.top
                player[0].landed()
                player[-1] = True
            elif dy < 0:
                player[0].rect.top = obj.rect.bottom
                player[0].hit_head()
                player[-1] = True

        collided_objects.append(obj)

    return collided_objects

def start_screen(window, background, bg_image):
    clock = pygame.time.Clock()
    font_title = pygame.font.Font(None, 70)
    font_subtitle = pygame.font.Font(None, 45)
    run = True
    show_instructions = False

    while run: 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                break

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    run = False
                elif event.key == pygame.K_i:
                    show_instructions = instruction_screen(window, background, bg_image)

        window.fill((187, 255, 255))
        title_text = font_title.render("Block Runner", True, (255, 48, 48))
        subtitle_text = font_subtitle.render("Press Space to Start, Press 'i' for instructions", True, (255, 48, 48))

        window.blit(title_text, (WIDTH // 2 - title_text.get_width() // 2, HEIGHT // 4 - title_text.get_height() // 2))
        window.blit(subtitle_text, (WIDTH // 2 - subtitle_text.get_width() // 2, HEIGHT // 2 - subtitle_text.get_height() // 2))

        # if show_instructions:
        #     instructions_text = font.render("Press Esc to go back", True, (0, 0, 0))
        #     window.blit(text, (WIDTH // 2 - text.get_width() // 2 - instructions_text.get_width() // 2, HEIGHT - 50))
            
        pygame.display.flip()
        clock.tick(FPS)

    return not show_instructions

def instruction_screen(window, background, bg_image):
    clock = pygame.time.Clock()
    font = pygame.font.Font(None, 36)
    run = True

    while run: 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                break

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    run = False

        for tile in background:
            window.blit(bg_image, tile)

        # window.fill((187, 255, 255))
        words = ["Instructions:", "Use ASWD to move the player", "Use arrow keys to move blocks", 
                 "Space to jump", "Press Esc to go back to title screen"]
        x = 50
        y = 50
        
        for i in words:
            text = font.render(i, True, (255, 48, 48))
            width, height = text.get_size()
            window.blit(text, (x, y))
            y += height + 25

        
        # instructions_text = font.render(["Instructions: Use arrow keys to move", "Space to jump", "Press Esc to go back"], True, (0, 0, 0))
        # window.blit(instructions_text, (WIDTH // 2 - instructions_text.get_width() // 2, HEIGHT // 2 - instructions_text.get_height() // 2))

        pygame.display.flip()
        clock.tick(FPS)

    return False

def scrolling(blocks, player, floor):
    if len(blocks) > 3:
        for i in range(len(blocks)):
            blocks[i].rect.y += 1
            for i in range(len(floor)):
                floor[i].rect.y += 0.5

    for j in range(len(blocks)):
        if blocks[j].rect.y >= HEIGHT:
            blocks[j].remove
    
    if player.rect.y >= HEIGHT:
#         game_over_screen(window, background, bg_image)
        pygame.quit()

def game_over_screen(window, background, bg_image):
    clock = pygame.time.Clock()
    font_title = pygame.font.Font(None, 70)
    font_subtitle = pygame.font.Font(None, 45)
    run = True
    show_overscreen = False
    pygame.display.flip()

    while run: 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                break

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    pygame.quit()
                    quit()
#                 elif event.key == pygame.K_r:
#                     return start_screen(window, background, bg_image)

    window.fill((187, 255, 255))
    title_text = font_title.render("Game Over" , True, (255, 48, 48))
    subtitle_text = font_subtitle.render("Press Q to Quit", True, (255, 48, 48))

    window.blit(title_text, (WIDTH // 2 - title_text.get_width() // 2, HEIGHT // 4 - title_text.get_height() // 2))
    window.blit(subtitle_text, (WIDTH // 2 - subtitle_text.get_width() // 2, HEIGHT // 2 - subtitle_text.get_height() // 2))

#     pygame.display.flip()
    clock.tick(FPS)
        
def main(window): 
    #setting up the game clock 
    clock = pygame.time.Clock()
    #getting a backgroung image 
    background, bg_image = get_background("Blue.png")
    show_instructions = start_screen(window, background, bg_image)
#     show_overscreen = game_over_screen(window, background, bg_image)

    block_size = 96

    block_spawn_timer = 0
    block_spawn_interval = 100
    
    player = Player(100, 100, 50, 50)
    #intitial falling block
    fall = FallingBlock(50, 200, 50, 50)

    #creating a floor made of blocks 
    floor = [Block(i * block_size, HEIGHT - block_size, block_size) for i in range(-WIDTH // block_size, (WIDTH * 2) // block_size)]

    #creates a list of game objects including the floor 
    objects =[*floor]
             
    #initializing a list of falling objects 
    fallingobjects = []

    #running the game loop
    run = True
    while run:
        #Control the frame rate of the game 
        clock.tick(FPS)

        #updating the block spawn timer 
        block_spawn_timer += 1 

        #spawning a new block when the block spawn interval passes the limit
        if block_spawn_timer >= block_spawn_interval:

            new_fall3 = generate_tetris_block(block_size)
           
            fallingobjects.append(new_fall3)
            
            block_spawn_timer = 0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                break

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and player.jump_count < 2:
                    player.jump()

        fall_len = len(fallingobjects)
        #checking if there is at least one falling block
        if fall_len != 0: 
            #the status of the last falling block is False
            if not(fallingobjects[-1][0][-1]) and not(fallingobjects[-1][1][-1]) and not(fallingobjects[-1][2][-1]) and not(fallingobjects[-1][3][-1]):
            
                #I have to individually handle the loop 
                curr_block1 = fallingobjects[-1][0]
                curr_block2 = fallingobjects[-1][1]
                curr_block3 = fallingobjects[-1][2]
                curr_block4 = fallingobjects[-1][3]
            
                curr_block1[0].loop(FPS)
                curr_block2[0].loop(FPS)
                curr_block3[0].loop(FPS)
                curr_block4[0].loop(FPS)
                    
                handle_move_forFallingBlock(curr_block1, objects)
                handle_move_forFallingBlock(curr_block2, objects)
                handle_move_forFallingBlock(curr_block3, objects)
                handle_move_forFallingBlock(curr_block4, objects)
                
                
                #curr_blocks[0][0].loop(FPS)
                #curr_block2[0][0].loop(FPS)

                #handle_move_forFallingBlock(curr_block1, objects)
                #handle_move_forFallingBlock(curr_block2, objects)

            #object 4 to handle move 
                objects4 = []
                for i in range(0, len(fallingobjects)-1):
                    objects4.append(fallingobjects[i][0][0])
                    objects4.append(fallingobjects[i][1][0])
                    objects4.append(fallingobjects[i][2][0])
                    objects4.append(fallingobjects[i][3][0])
                    #objects4.append(fallingobjects[i][0][0])
                    #objects4.append(fallingobjects[i][1][0])
               
                handle_move_forFallingBlock(curr_block1, objects4)
                handle_move_forFallingBlock(curr_block2, objects4)
                handle_move_forFallingBlock(curr_block3, objects4)
                handle_move_forFallingBlock(curr_block4, objects4)

        #runnng loop for player 
        player.loop(FPS)
        #handle move with player and objects 
        handle_move(player, objects)

        #handle move between player and all blocks 
        objects3 = []
        for i in fallingobjects:
            objects3.append(i[0][0])
            objects3.append(i[1][0])
            objects3.append(i[2][0])
            objects3.append(i[3][0])
        handle_move(player, objects3)

        
#         draw(window, background, bg_image, player, objects, objects3)
        draw(window, background, bg_image, player, objects, objects3, instructions = show_instructions)
        scrolling(objects3, player, floor)
#         if len(objects3) > 3:
#             for i in range(len(objects3)):
#                 objects3[i].rect.y += 1
#                 for i in range(len(floor)):
#                     floor[i].rect.y += 0.5

#             for j in range(len(objects3)):
#                 if objects3[j].rect.y >= HEIGHT:
#                     objects3[j].remove
    
#             if player.rect.y >= HEIGHT:
#                 print("FELL")
#                 game_over_screen(window, background, bg_image)
#                 run = False
        
    pygame.quit()
#     print("exit")
    quit()
#     game_over_screen(window, background, bg_image)

if __name__ == "__main__":
    main(window)

pygame 2.5.2 (SDL 2.28.3, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: video system not initialized